In [0]:
storage_account_name = "mlopsdatastore"
container_name = "titanic"
storage_account_key = "Hw6MPel/fLHOa+OiqsptrrbPrLniRp0zFc9lwPjC+sJEQk1YbR0oS25uFFMLCEuKCoJzq1ZleAJP+AStftHVAg=="

In [0]:
# Set the Spark configuration
spark.conf.set(
  "fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account_name),
  storage_account_key
)

# Mount the Blob Storage containerr
dbutils.fs.mount(
  source = "wasbs://{0}@{1}.blob.core.windows.net".format(container_name, storage_account_name),
  mount_point = "/mnt/datamount/data",
  extra_configs = {"fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account_name): storage_account_key}
)

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-1896740026641052>, line 8
      2 spark.conf.set(
      3   "fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account_name),
      4   storage_account_key
      5 )
      7 # Mount the Blob Storage container
----> 8 dbutils.fs.mount(
      9   source = "wasbs://{0}@{1}.blob.core.windows.net".format(container_name, storage_account_name),
     10   mount_point = "/mnt/datamount/data",
     11   extra_configs = {"fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account_name): storage_account_key}
     12 )

File /databricks/python_shell/dbruntime/dbutils.py:362, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    360 exc.__context__ = None
    361 exc.__cause__ = None
--> 362 raise exc

ExecutionError: An error occurred while calling o1466.mount.


In [0]:
# Define the file path
file_path = "/mnt/datamount/data/train.csv"

# Read the CSV file into a DataFrame
train_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(file_path)

# train_df.write.format("delta").save("/mnt/datamount/titanic_delta")

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.window import Window
from pyspark.sql import functions as F


# Create a Spark session
spark = SparkSession.builder.appName("PandasToPySpark").getOrCreate()

#'train_df' is a PySpark DataFrame
train_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/datamount/data/train.csv")

# Drop the 'Cabin' column
train_df = train_df.drop('Cabin')

# Drop rows with missing values
train_df = train_df.na.drop()

# Drop duplicates
train_df = train_df.dropDuplicates()

# Calculate the Z-scores for 'Fare' column
w = Window.orderBy(F.lit(0))
train_df = train_df.withColumn("FareZScore", F.abs(F.col("Fare") - F.avg("Fare").over(w)) / F.stddev("Fare").over(w))

# Set a Z-score threshold for outlier removal
threshold = 0.1

# Create a new DataFrame with outliers removed
df_no_outliers = train_df.filter(train_df['FareZScore'] <= threshold).drop("FareZScore")

# Show the resulting DataFrame
df_no_outliers.show()

+-----------+--------+------+--------------------+------+----+-----+-----+---------------+-------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|         Ticket|   Fare|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+---------------+-------+--------+
|        542|       0|     3|Andersson, Miss. ...|female| 9.0|    4|    2|         347082| 31.275|       S|
|        450|       1|     1|Peuchen, Major. A...|  male|52.0|    0|    0|         113786|   30.5|       S|
|        849|       0|     2|   Harper, Rev. John|  male|28.0|    0|    1|         248727|   33.0|       S|
|        648|       1|     1|Simonius-Blumer, ...|  male|56.0|    0|    0|          13213|   35.5|       C|
|         26|       1|     3|Asplund, Mrs. Car...|female|38.0|    1|    5|         347077|31.3875|       S|
|        721|       1|     2|"Harper, Miss. An...|female| 6.0|    0|    1|         248727|   33.0|       S|
|        171|       0|     1

In [0]:
# Write the streaming DataFrame to the Delta table
df_no_outliers.write.format("delta").mode("overwrite").save("/mnt/datamount/delta_table")

In [0]:
dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()

'/Users/bhaveshkak26122000@gmail.com/ETL'